In [1]:
library(survey)
library(tidyverse)

Loading required package: grid
Loading required package: Matrix
Loading required package: survival

Attaching package: ‘survey’

The following object is masked from ‘package:graphics’:

    dotchart

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.3.4     ✔ dplyr   0.7.4
✔ tidyr   0.7.2     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand() masks Matrix::expand()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
data_dir <- file.path("..", "data")

In [3]:
full_data <- readRDS(file.path(data_dir, "full_data_ps.rds"))
head(full_data)

icustay_id,hadm_id,subject_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,⋯,lab_ph_flag,lab_ph_first,lab_ph_min,lab_ph_max,lab_ph_abnormal,sedative,echo_int,mort_28_day_int,ps,ps_weight
201220,125078,66690,0,2106-04-27 01:47:50,2106-05-01 11:25:46,1,62.67646,1,NA,⋯,1,7.45,7.42,7.45,1,1,0,1,0.4544669,1.833069
215842,151232,11663,0,2188-02-14 01:48:15,2188-02-15 19:02:48,1,86.76186,1,NA,⋯,0,NA,NA,NA,NA,0,0,0,0.1568838,1.186076
234312,164444,86645,1,2165-06-22 01:47:16,2165-07-07 13:55:20,1,56.08904,1,NA,⋯,1,7.49,7.49,7.49,1,0,0,0,0.3002571,1.429096
289157,146726,10304,0,2156-06-23 14:26:00,2156-06-30 09:26:00,1,45.91093,1,2156-06-25,⋯,1,7.44,7.44,7.44,1,0,1,0,0.6991412,1.430326
211964,160170,94534,0,2160-03-05 14:23:19,2160-03-06 22:48:41,1,59.38693,1,NA,⋯,0,NA,NA,NA,NA,0,0,0,0.3432285,1.522600
230173,112553,31544,0,2140-01-30 20:39:25,2140-02-02 18:41:39,1,91.50000,1,NA,⋯,0,NA,NA,NA,NA,0,0,1,0.2464983,1.327137


In [4]:
table_one <- data.table::fread(file.path(data_dir, "tableone_raw.csv"), data.table = FALSE)
head(table_one)

p_val,wtd_p_val,main_0,main_1,sub_0,sub_1,wtd_main_0,wtd_main_1,wtd_sub_0,wtd_sub_1,var
4.472893e-02,7.861962e-02,66.689241,65.824051,17.207039,16.620088,65.444828,64.654880,17.014068,16.466339,age
1.416420e-02,1.507728e-02,50.822846,47.665687,NA,NA,47.089492,43.841849,NA,NA,female
6.415559e-11,1.909584e-13,78.562949,82.979049,23.579777,26.702495,85.637556,91.568834,27.685863,32.234984,weight
1.705518e-15,4.041212e-14,8.505970,10.045054,7.451124,7.681609,8.332934,9.888314,7.564772,7.744827,elix_score
1.378591e-15,8.274714e-12,19.632462,20.778975,5.788347,5.446293,19.793740,20.807581,5.667359,5.364432,saps
7.217556e-27,0.000000e+00,5.304937,6.323865,3.618100,3.796365,5.528346,6.510264,3.764854,3.812443,sofa


In [5]:
wpv <- table_one %>%
    filter(wtd_p_val < 0.05) %>%
    pull(var) %>%
    gsub("female", "gender", .)

if (length(grep("day", wpv)) > 0) wpv <- wpv %>% discard(grepl("day", .)) %>% c(., "icu_adm_weekday")

wpv

[1] "gender"                     "weight"                    
 [3] "elix_score"                 "saps"                      
 [5] "sofa"                       "vent"                      
 [7] "vaso"                       "sedative"                  
 [9] "icd_chf"                    "icd_afib"                  
[11] "icd_renal"                  "icd_copd"                  
[13] "icd_cad"                    "icd_stroke"                
[15] "icd_malignancy"             "vs_heart_rate_first"       
[17] "vs_temp_first"              "lab_platelet_first"        
[19] "lab_sodium_first"           "lab_potassium_first"       
[21] "lab_chloride_first"         "lab_bun_first"             
[23] "lab_creatinine_first"       "lab_po2_first"             
[25] "lab_pco2_first"             "lab_bnp_flag"              
[27] "lab_troponin_flag"          "lab_creatinine_kinase_flag"
[29] "icu_adm_weekday"

In [6]:
design_echo_ps_ate <- svydesign(ids = ~ icustay_id, weights = ~ ps_weight, data = full_data)

In [7]:
fml <- wpv %>%
c("echo", .) %>%
paste(collapse = " + ") %>%
sprintf("mort_28_day ~ %s", .)

fml

[1] "mort_28_day ~ echo + gender + weight + elix_score + saps + sofa + vent + vaso + sedative + icd_chf + icd_afib + icd_renal + icd_copd + icd_cad + icd_stroke + icd_malignancy + vs_heart_rate_first + vs_temp_first + lab_platelet_first + lab_sodium_first + lab_potassium_first + lab_chloride_first + lab_bun_first + lab_creatinine_first + lab_po2_first + lab_pco2_first + lab_bnp_flag + lab_troponin_flag + lab_creatinine_kinase_flag + icu_adm_weekday"

In [8]:
logi <- svyglm(as.formula(fml), family = quasibinomial, design = design_echo_ps_ate)

In [9]:
summary(logi)


Call:
svyglm(formula = as.formula(fml), family = quasibinomial, design = design_echo_ps_ate)

Survey design:
svydesign(ids = ~icustay_id, weights = ~ps_weight, data = full_data)

Coefficients:
                              Estimate Std. Error t value Pr(>|t|)    
(Intercept)                 -1.1375315  1.7778174  -0.640 0.522313    
echo1                       -0.3187552  0.0920163  -3.464 0.000538 ***
gender1                      0.1325798  0.0936064   1.416 0.156758    
weight                      -0.0113965  0.0020855  -5.465 4.95e-08 ***
elix_score                   0.0297429  0.0067464   4.409 1.07e-05 ***
saps                         0.0773929  0.0114177   6.778 1.41e-11 ***
sofa                         0.1687837  0.0172009   9.812  < 2e-16 ***
vent1                        0.1029206  0.1537603   0.669 0.503310    
vaso1                        0.0528519  0.1066875   0.495 0.620355    
sedative1                   -0.3932804  0.1300448  -3.024 0.002511 ** 
icd_chf1                 

In [10]:
exp(cbind(OR = coef(logi), confint(logi)))

,OR,2.5 %,97.5 %
(Intercept),0.3206095,0.009833213,10.4533930
echo1,0.7270535,0.607074447,0.8707445
gender1,1.1417701,0.950387677,1.3716918
weight,0.9886682,0.984635358,0.9927176
elix_score,1.0301896,1.016657391,1.0439020
saps,1.0804665,1.056556056,1.1049180
sofa,1.1838640,1.144617509,1.2244563
vent1,1.1084034,0.820005752,1.4982312
vaso1,1.0542735,0.855343819,1.2994688
sedative1,0.6748395,0.523005373,0.8707528
